# 02g Co-rutinas

Una función común o sub-rutina tiene un único punto de entrada y un único punto de salida. Al finalizar la ejecución de la función, el programa olvida el estado de todas las variables declaradas en su interior.

Un [generador](02g.ipynb) tiene un único punto de entrada, pero múltiples puntos de salida. La ejecución se suspende y se puede volver a reanudar, manteniendo el estado de las variables internas.

Una co-rutina tiene múltiples puntos de entrada y salida. Su ejecución se puede suspender y reanudar, y el estado de las variables se mantiene, y se puede actualizar desde el exterior.

Para obtener un valor desde el exterior de la co-rutina, se usa la expresión `yield` (esta vez, en el lado derecho de un `=`). Cuando el programa llega a una expresión con `yield`, la co-rutina se suspende hasta que reciba un valor:

In [1]:
def imprime_recibido():
    print("Iniciando la co-rutina...")
    while 1:
        recibido = (yield)
        print(f"He recibido {recibido}!")

Los valores se pasan a la co-rutina mediante su método `.send()`:

In [2]:
imprimir = imprime_recibido()
imprimir.send(5)

TypeError: can't send non-None value to a just-started generator

¡Pero antes de poder enviar nada, hay que iniciar la co-rutina enviando un mensaje `None`!:

In [3]:
imprimir = imprime_recibido()
imprimir.send(None)
imprimir.send(5)

Iniciando la co-rutina...
He recibido 5!


In [4]:
imprimir.send(27)

He recibido 27!


Para evitar olvidar la iniciación, se suele usar un decorador que se ocupe de ello:

In [5]:
def corutina(func):
    def iniciada(*args, **kwargs):
        cor = func(*args, **kwargs)
        cor.send(None)
        return cor
    return iniciada

@corutina
def imprime_recibido():
    print("Iniciando la co-rutina...")
    while 1:
        recibido = (yield)
        print(f"He recibido {recibido}!")


imprimir = imprime_recibido()
imprimir.send(8)

Iniciando la co-rutina...
He recibido 8!


Veamos un ejemplo en el que la co-rutina modifica su estado interno:

In [6]:
@corutina
def suma(x=0):
    valor = x
    while 1:
        x += (yield)
        print(f"Valor de la suma: {x}")

In [7]:
mi_suma = suma()
mi_suma.send(7)
mi_suma.send(10)
mi_suma.send(-3)

Valor de la suma: 7
Valor de la suma: 17
Valor de la suma: 14


El `yield` en realidad realiza dos tareas al mismo tiempo: captura los valores enviados por `.send()`, pero también pueden sacar valores al exterior de la co-rutina, tal y como lo hacía en los generadores:

In [8]:
@corutina
def suma(x=0):
    valor = x
    while 1:
        x += (yield x)

In [9]:
mi_suma = suma()
print(mi_suma.send(7))
print(mi_suma.send(10))
print(mi_suma.send(-3))

7
17
14


Sin embargo, hay que tener cuidado si se usa una co-rutina, ya que hacerle `next()` (por ejemplo, en un bucle `for`), es equivalente a `.send(None)`:

In [10]:
mi_suma = suma()
for i, s in zip(range(1, 8), mi_suma):
    print(s)
    mi_suma.send(i)

TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'

Modificamos la co-rutina para evitar los valores `None`:

In [11]:
@corutina
def suma(x=0):
    valor = x
    while 1:
        rec = (yield x)
        if rec is not None:
            x += rec

In [12]:
mi_suma = suma()
for i, s in zip(range(1, 8), mi_suma):
    print(s)
    mi_suma.send(i)

0
1
3
6
10
15
21


La co-rutina se cierra con el método `close()`.

In [13]:
mi_suma.close()

## Tuberías

Igual que hicimos con los generadores, también podemos encadenar co-rutinas para procesar una secuencia de datos. En este caso, el flujo de la ejecución va desde la fuente hacia el sumidero. Por ello, a cada co-rutina le tendremos que pasar como argumento la co-rutina de destino:

In [14]:
def envia_lineas(archivo, corut):
    with open(archivo, "rt") as f:
        for line in f:
            corut.send(line)

@corutina
def separa_comas(corut):
    while 1:
        line = (yield)
        corut.send(line.strip().split(","))

@corutina
def vector(corut):
    while 1:
        s = (yield)
        corut.send((int(s[0]), int(s[1]), int(s[2])))

@corutina
def modulos(corut):
    while 1:
        v = (yield)
        corut.send((v[0]**2+v[1]**2+v[2]**2)**0.5)

@corutina
def modulos_menor_10(corut):
    while 1:
        m = (yield)
        if m < 10:
            corut.send(m)

@corutina
def contar():
    n = 0
    while 1:
        m = (yield n)
        if m is not None:
            n += 1
    yield n

Para obtener el resultado, en la última co-rutina además de recibir datos también los trasmitimos con `(yield n)`. Como el contador solamente se actualiza si se ha recibido un valor que no es `None`, podemos consultar el resultado usando `next()` (o `.send(None)`):

In [15]:
c = contar()

envia_lineas("input_02c", separa_comas(vector(modulos(modulos_menor_10(c)))))

next(c)

98

o escrito de una forma ligeramente más legible:

In [16]:
c = contar()
parte1 = modulos_menor_10(c)
parte2 = modulos(parte1)
parte3 = vector(parte2)
parte4 = separa_comas(parte3)

envia_lineas("input_02c", parte4)

next(c)

98

A diferencia del caso de los generadores, que no son reutilizables, las co-rutinas permiten un procesado más complejo, en el que se creen ramas si una co-rutina envía datos a varias co-rutinas:

In [17]:
@corutina
def modulos_menor_10(corutinas):
    while 1:
        m = (yield)
        if m < 10:
            for corut in corutinas:
                corut.send(m)

@corutina
def producto():
    x = 1
    while 1:
        m = (yield x)
        if m is not None:
            x *= m

In [18]:
c = contar()
p = producto()
parte1 = modulos_menor_10([c, p])
parte2 = modulos(parte1)
parte3 = vector(parte2)
parte4 = separa_comas(parte3)

envia_lineas("input_02c", parte4)

print(f"contar: {next(c)}")
print(f"producto: {next(p)}")

contar: 98
producto: 6.481033952968496e+93
